In [1]:
import numpy as np
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

model = models.CellposeModel(gpu=True)



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


2025-08-25 14:26:13,980 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-08-25 14:26:13,980 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-08-25 14:26:14,113 [INFO] ** TORCH CUDA version installed and working. **
2025-08-25 14:26:14,113 [INFO] ** TORCH CUDA version installed and working. **
2025-08-25 14:26:14,113 [INFO] >>>> using GPU (CUDA)
2025-08-25 14:26:14,991 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam


In [2]:
from pathlib import Path
import napari
from utils import list_images, read_image

In [3]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("./raw_data/test_data_2")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

['raw_data\\test_data_2\\Plate01_Nuc_Wells-A1__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'raw_data\\test_data_2\\Plate01_Nuc_Wells-A2__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'raw_data\\test_data_2\\Plate01_Nuc_Wells-A3__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'raw_data\\test_data_2\\Plate01_Nuc_Wells-A4__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2']

In [4]:
# Explore a different image to crop (0 defines the first image in the directory)
image = images[1]

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)



Image analyzed: Plate01_Nuc_Wells-A2__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR
Original Array shape: (6, 5032, 5032)
Compressed Array shape: (6, 5032, 5032)


In [5]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)

<Image layer 'img' at 0x1aa385024a0>

In [6]:
nuclei_labels, flows, styles = model.eval(img[-1:], niter=1000) # need to check the arguments
viewer.add_labels(nuclei_labels)

<Labels layer 'nuclei_labels' at 0x1ab11800e50>

In [7]:
cytoplasm_labels, flows, styles = model.eval(np.stack((img[[0,1]].sum(axis=0), img[4]), axis=0), niter=1000) # need to check the arguments
viewer.add_labels(cytoplasm_labels)

<Labels layer 'cytoplasm_labels' at 0x1aa384bccd0>

In [8]:
from spotiflow.model import Spotiflow

c:\Users\adiez_cmic\miniforge3\envs\hcs_cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
spotiflow_model = Spotiflow.from_pretrained("general")

INFO:spotiflow.model.spotiflow:Loading pretrained model: general
2025-08-25 14:40:35,796 [INFO] Loading pretrained model: general


In [ ]:
points, details = spotiflow_model.predict(img[0], subpix=True)
viewer.add_image(details.heatmap, colormap="viridis")
viewer.add_points(points, face_color='red')

INFO:spotiflow.model.spotiflow:Will use device: cuda:0
2025-08-25 14:41:21,543 [INFO] Will use device: cuda:0
INFO:spotiflow.model.spotiflow:Predicting with prob_thresh = [0.6], min_distance = 1
2025-08-25 14:41:21,543 [INFO] Predicting with prob_thresh = [0.6], min_distance = 1
INFO:spotiflow.model.spotiflow:Peak detection mode: fast
2025-08-25 14:41:21,545 [INFO] Peak detection mode: fast
INFO:spotiflow.model.spotiflow:Image shape (5032, 5032)
2025-08-25 14:41:21,545 [INFO] Image shape (5032, 5032)
INFO:spotiflow.model.spotiflow:Predicting with (3, 3) tiles
2025-08-25 14:41:21,547 [INFO] Predicting with (3, 3) tiles
INFO:spotiflow.model.spotiflow:Normalizing...
2025-08-25 14:41:21,588 [INFO] Normalizing...
INFO:spotiflow.model.spotiflow:Padding to shape (5040, 5040, 1)
2025-08-25 14:41:21,822 [INFO] Padding to shape (5040, 5040, 1)


Predicting tiles: 100%|██████████| 9/9 [00:03<00:00,  2.85it/s]

INFO:spotiflow.model.spotiflow:Found 3341 spots
2025-08-25 14:41:25,021 [INFO] Found 3341 spots


<Image layer 'Image' at 0x1ab4bd2d360>